# Writing Our First Classifier

## Introduction

Bu bölümde kendi sınıflandırıcımızı yazacağız ve sınıflandırıcı, k-Nearest Neighbors'ın hurda bir versiyonu olack. Bu, en basit sınıflandırıcılardan biridir. Bölüm 4'te, bir veri setini içe aktardık ve onu train/test'e ayırdık. Bir sınıflandırıcıyı eğitmek için train kullandık ve ne kadar doğru olduğunu görmek için test ettik.

Önceden sınıflandırıcıyı bu iki satırı kullanarak bir kitaplıktan içe aktarıyorduk.

```py
from sklearn.neighbors import KNeighborsClassifier
my_classifier = KNeighborsClassifier()
```

Şimdi onları yorumlayıp kendi sınıflandırıcımızı yazacağız. Pipeline'nın  geri kalanı tamamen aynı kalacak. İris datası için bu sınıflandırıcın doğruluk oranı %90'nın üzerindeydi. Bu oranı yakalayabileceğimiz, kendi sınıflandırıcımızı yazmayı amaçlıyoruz.

Yapmamız gereken ilk şey pipeline'ı düzeltmek...

## Implement a class

Sınıflandırıcımız için ScrappyKNN adında bir sınıf uygulayacağız. Sınıfımız **fit** ve **predict** davranşlarına sahip olacak.
fit metodumuz, training setimizin features ve label'larını girdi olarak alacak. **predict** yöntemimiz ise girdi olarak test verilerimizin özelliklerini alır ve çıktı olarak, *label* döndürür. Şimdi rastgele bir sınıflandırıcı yazacağız.

In [1]:
import random

class ScrappyKNN():
    
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def predict(self, X_test):
        predictions = []
        for row in X_test:
            label = random.choice(self.y_train)
            predictions.append(label)
        return predictions

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()

X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5)

my_classifier = ScrappyKNN()
my_classifier.fit(X_train, y_train)

y_pred = my_classifier.predict(X_test)

İris veri setinde üç farklı çiçek türü olduğunu hatırlayın, bu nedenle doğruluk yaklaşık% 33 olmalıdır. 

In [3]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.38666666666666666

## Intro to k-NN

Ekranda gördüğümüz noktaların, fit metoduyla ezberlediğimiz eğitim verileri olduğunu hayal edin.

<img src = 'https://raw.githubusercontent.com/birdalugureren/Machine-Learning-Recipes/master/img/knn1.png?token=AGJJGUDR5SV7U5OJKMB7HM276I5KS'>

Şimdi, gri olarak belirtilen bu test noktası için bir tahmin yapmamızın istendiğini hayal edin. Bunu nasıl yapabiliriz?

k-NN (En yakın komşu), tam olarak göründüğü gibi çalışır yani test noktasına en yakın eğitim noktasını bulacağız sonra test noktasının aynı etikete sahip olduğunu tahmin edeceğiz. Bu örnekte *yeşil*.

Bu örnekte ise nokta, en yakın yeşil noktaya ve en yakın kırmızı noktaya eşit uzaklıkta. Bir yol, beraberliği rastgele bozabilmemiz. Başka bir yol ise (k burada devreye girer) tahminimizi yaparken dikkate aldığımız komşuların sayısıdır.
k=3 olsaydı, en yakın üçüne bakardık. Bu durumda, bunlardan ikisi yeşil ve biri kırmızıdır.

<img src = 'https://raw.githubusercontent.com/birdalugureren/Machine-Learning-Recipes/master/img/knn3.png?token=AGJJGUGZP57IPHELRCESAGS76I5OA'>

## Measure distance

En yakın komşuyu bulmak için iki nokta arasındaki düz çizgi mesafesini ölçeceğiz, tıpkı bir cetvelle yaptığınız gibi.
Bunun için **Öklid Mesafesi** denen bir formül var ve işte formül şöyle görünüyor.

<img src='https://raw.githubusercontent.com/birdalugureren/Machine-Learning-Recipes/master/img/oclid.png?token=AGJJGUDJNWLVG7J43N76FUC76I5P6'>

İki nokta arasındaki mesafeyi ölçer. Hesapladığımız mesafe hipotenüsün uzunluğudur.

Şu anda iki boyutlu uzayda mesafeyi hesaplıyoruz çünkü veri setimizde sadece iki özelliğimiz var. Peki ya üç özelliğimiz veya üç boyutumuz olsaydı? O zaman bir küpün içinde olurduk. Uzaydaki mesafenin bir cetvelle nasıl ölçüleceğini hala görselleştirebiliriz. Peki ya iriste yaptığımız gibi dört özelliğimiz veya dört boyutumuz olsaydı? Şimdi bir hypercube'teyiz.

<img src='https://raw.githubusercontent.com/birdalugureren/Machine-Learning-Recipes/master/img/hypercube.png?token=AGJJGUDNIYRZJALJN7VCW3K76I5RG'>

İyi haber, Öklid Mesafesinin boyutların sayısına bakılmaksızın aynı şekilde çalışmasıdır. Daha fazla özellikle, denkleme daha fazla terim ekleyebiliriz.

\begin{equation*}
d(a,b) = \sqrt{(x_{2}-x_{1})^2 + (y_{2}-y_{1})^2 + ... +(n_{2}-n_{1})^2}
\end{equation*}

Şimdi Öklid mesafesini kodlayalım. **scipy** adlı bir kitaplık kullanacağız. Burada, a ve b sayısal özelliklerin listesidir. a, eğitim verilerimizden bir nokta ve b, test verilerimizden bir nokta.

In [4]:
from scipy.spatial import distance

def euc(a,b):
    """
    Returns the distance between them."""
    return distance.euclidean(a,b)

## Implement nearest neighbor algorithm

Şimdi bir sınıflandırıcı için algoritmaya bir göz atalım. Tüm eğitim noktalarına olan mesafeyi hesaplayacağız. Sonra test noktasının en yakın olanla aynı etikete sahip olduğunu tahmin edeceğiz. Yaptığımız rastgele tahmini sileceğiz ve onu test noktasına en yakın eğitim noktasını bulan bir yöntemle değiştireceğiz. Bu eğitim için k=1 sabit tutacağız.

In [5]:
class ScrappyKNN():
    
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def predict(self, X_test):
        predictions = []
        for row in X_test:
            label = self.closest(row)
            predictions.append(label)
        return predictions
    
    def closest(self, row):
        best_dist = euc(row, self.X_train[0])
        best_index = 0
        for i in range(1, len(self.X_train)):
            dist = euc(row, self.X_train[i])
            if dist < best_dist:
                best_dist = dist
                best_index = i
        return self.y_train[best_index]

In [6]:
## Run pipeline

In [7]:
my_classifier = ScrappyKNN()
my_classifier.fit(X_train, y_train)

y_pred = my_classifier.predict(X_test)

In [8]:
accuracy_score(y_test, y_pred)

0.96